# Create samples from the training data

The goal of this notebook is to make samples out of the training data provided by the organizers

In [8]:
import os, sys
import pandas as pd
import numpy as np
from util import *
import pycs

In [12]:
# load the data
datapath = "data/training_set.csv"
metadata = "data/training_set_metadata.csv"

df = pd.read_csv(datapath)
dfm = pd.read_csv(metadata)

print dfm.tail()

      object_id          ra       decl       gal_l      gal_b  ddf  \
7843  130739978   26.718750 -14.940303  172.342697 -72.255675    0   
7844  130755807  120.101349 -62.696659  275.742955 -16.509746    0   
7845  130762946  203.108109 -55.682144  308.728904   6.727511    0   
7846  130772921   79.101562 -35.501846  239.172243 -33.827844    0   
7847  130779836  301.992188 -17.426323   25.102988 -24.511101    0   

      hostgal_specz  hostgal_photoz  hostgal_photoz_err  distmod  mwebv  \
7843         0.0000          0.0000              0.0000      NaN  0.013   
7844         0.1725          2.5606              1.1146  46.6108  0.136   
7845         0.0000          0.0000              0.0000      NaN  0.430   
7846         0.0000          0.0000              0.0000      NaN  0.034   
7847         0.0000          0.0000              0.0000      NaN  0.091   

      target  
7843      65  
7844      90  
7845      16  
7846      65  
7847       6  


In [13]:
indexes = np.unique(df["object_id"])
bands = np.unique(df["passband"])

# in case we want to map back go the real filters
bands_mapping = {0: "u", 1: "g", 2: "r", 3: "i", 4: "z", 5: "y"}
# df["passband"].map(bands_mapping)


print dfm.head(1).T
print "\n\n"
print df.head(1).T

                             0
object_id           615.000000
ra                  349.046051
decl                -61.943836
gal_l               320.796530
gal_b               -51.753706
ddf                   1.000000
hostgal_specz         0.000000
hostgal_photoz        0.000000
hostgal_photoz_err    0.000000
distmod                    NaN
mwebv                 0.017000
target               92.000000



                      0
object_id    615.000000
mjd        59750.422900
passband       2.000000
flux        -544.810303
flux_err       3.622952
detected       1.000000


In [14]:
from tqdm import tqdm  # fancy progress prints

series_list = []
for i in tqdm(range(len(dfm))):
	dfitem = dfm.iloc[i]
	object_id = dfitem["object_id"]
	
	# pandas' not very concilient with appending data to a Series or dataframe 
	# (or I'm not good with the doc) 
	# --> I build a dictionnary that I re-translate in a Series later
	sample = dfitem.to_dict()

	# build light curves, looking at the row of df
	# that have the correct passband and id
	# it would probably be faster to loop on df lines than looking for the good ones at each step...to keep in mind when working with the test set
	for b in bands:
		sample["mjds_%s" % b] = df["mjd"][df["passband"] == b][df["object_id"] == object_id].values
		sample["fluxes_%s" % b] = df["flux"][df["passband"] == b][df["object_id"] == object_id].values
		sample["fluxerrs_%s" % b] = df["flux_err"][df["passband"] == b][df["object_id"] == object_id].values
		sample["detected_%s" % b] = df["detected"][df["passband"] == b][df["object_id"] == object_id].values
	
		# now the spline fitting part...
		(mags, magerrs) = asinhmag(sample["fluxes_%s" % b], sample["fluxerrs_%s" % b])		
		lc = pycs.gen.lc.factory(jds=sample["mjds_%s" % b], mags=mags, magerrs=magerrs)
	
		# really rough test, we just fit a spline with chosen knotstep
		spl = pycs.gen.spl.fit(lcs=[lc], knotstep=20.0, stab=True,
	stabext=100.0, bokit=1, bokeps=2.0, boktests=5, verbose=False)	

		sample["spl_%s_t" % b] = spl.t
		sample["spl_%s_c" % b] = spl.c
	
	# create a Series with the dict and put that in a big list
	series_list.append(pd.Series(sample))
	
# create a dataframe out of the list	
samples = pd.concat(series_list, axis=1).T

# convert what needs to be integer as integer
# ooh look Im pandas Im so fucking smart I save everything in float lolilol
samples[["object_id", "target"]] = samples[["object_id", "target"]].astype(int)

  0%|          | 0/7848 [00:00<?, ?it/s]

  0%|          | 1/7848 [00:00<1:17:36,  1.69it/s]

  0%|          | 2/7848 [00:01<1:14:35,  1.75it/s]

  0%|          | 3/7848 [00:01<1:14:18,  1.76it/s]

  0%|          | 4/7848 [00:02<1:13:45,  1.77it/s]

...

100%|█████████▉| 7844/7848 [1:17:58<00:02,  1.68it/s]

100%|█████████▉| 7845/7848 [1:17:58<00:01,  1.68it/s]

100%|█████████▉| 7846/7848 [1:17:59<00:01,  1.68it/s]

100%|█████████▉| 7847/7848 [1:18:00<00:00,  1.68it/s]

100%|██████████| 7848/7848 [1:18:00<00:00,  1.68it/s]

In [16]:
# pickle the results for later use
writepickle(samples, "training_samples_spl.pkl")

Wrote training_samples_spl.pkl


In [17]:
samples = readpickle("training_samples_spl.pkl")
samples.iloc[654]

Read training_samples_spl.pkl


ddf                                                                   1
decl                                                           -46.7685
detected_0            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
detected_1            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
detected_2            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
detected_3            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
detected_4            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
detected_5            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
distmod                                                         41.6466
fluxerrs_0            [1.688356, 1.931612, 1.795236, 2.735116, 2.843...
fluxerrs_1            [0.619115, 2.848189, 1.178133, 0.971095, 3.389...
fluxerrs_2            [0.630288, 2.086514, 1.240831, 1.002995, 2.331...
fluxerrs_3            [1.065594, 2.487419, 2.088372, 1.705569, 2.485...
fluxerrs_4            [1.825245, 3.297901, 2.579784, 2.62245, 2.

In [40]:
# explore the days spacing

minmjd = min([min([min(samples.iloc[i]["mjds_%i" % bi]) for bi in range(6)]) for i in range(len(samples))])

maxmjd = max([max([max(samples.iloc[i]["mjds_%i" % bi]) for bi in range(6)]) for i in range(len(samples))])

print "minimum day: ", minmjd
print "maximum day: ", maxmjd
print "interval: ", maxmjd-minmjd

allmjds = []
for i in range(len(samples)):
	for bi in range(6):
		for val in samples.iloc[i]["mjds_%i" % bi]:
			allmjds.append(val)
			
		
print "Total number of days: ", len(allmjds)
print "Total number of unique days: ", len(np.unique(allmjds))




NameError: name 'samples' is not defined

In [48]:
# explore the flux spacing

minflux = min([min([min(samples.iloc[i]["fluxes_%i" % bi]) for bi in range(6)]) for i in range(len(samples))])

maxflux = max([max([max(samples.iloc[i]["fluxes_%i" % bi]) for bi in range(6)]) for i in range(len(samples))])

print "minimum flux: ", minflux
print "maximum flux: ", maxflux
print "interval: ", maxflux-minflux

allfluxes = []
for i in range(len(samples)):
	for bi in range(6):
		for val in samples.iloc[i]["fluxes_%i" % bi]:
			allfluxes.append(val)
			
		
print "Total number of fluxes: ", len(allfluxes)
print "Total number of positive fluxes: ", len([f for f in allfluxes if f  >0])
print "Total number of unique fluxes: ", len(np.unique(allfluxes))


NameError: name 'samples' is not defined

In [52]:
# Let's look a the fluxerrs now...

minfe = min([min([min(samples.iloc[i]["fluxerrs_%i" % bi]) for bi in range(6)]) for i in range(len(samples))])

maxfe = max([max([max(samples.iloc[i]["fluxerrs_%i" % bi]) for bi in range(6)]) for i in range(len(samples))])


print "minimum fluxerr: ", minfe
print "maximum fluxerr: ", maxfe
print "interval: ", maxfe-minfe


NameError: name 'samples' is not defined

In [3]:
# Playing with splines.

import pycs
from tqdm import tqdm

def asinhmag(flux, fluxerr, m0=22.5, f0=1.0, b=0.01):
	"""
	Implements asinh magnitudes, following
	http://ssg.astro.washington.edu/elsst/opsim.shtml?lightcurve_mags

	@param flux: numpy array of floats
	@param fluxerr: numpy array of floats
	@param m0: float. Zero-point of the magnitude system.
	@param f0: float. asinh mag normalisation constant.
	@param b: float. Asinh mag normalisation constant.
	@return: tuple containing two lists of fluxes converted into magnitudes
	"""

	mag = m0 - (2.5 / np.log(10.)) * (np.arcsinh(flux / (f0 * 2.0 * b)) + np.log(b))

	magplu = m0 - (2.5 / np.log(10.)) * (np.arcsinh((flux + fluxerr) / (f0 * 2.0 * b)) + np.log(b))
	magmin = m0 - (2.5 / np.log(10.)) * (np.arcsinh((flux - fluxerr) / (f0 * 2.0 * b)) + np.log(b))
	magerr = 0.5 * (magmin - magplu)
	

	return (mag, magerr)


for i in tqdm(range(len(samples))):
	for ib in range(6):

		#detecteds = [True if d>0 else False for d in samples.iloc[i]["detected_%i" % bi]]
		fluxes = samples.iloc[i]["fluxes_%i" % bi]
		fluxerrs = samples.iloc[i]["fluxerrs_%i" % bi]
		(mags, magerrs) = asinhmag(fluxes, fluxerrs)
		mjds = samples.iloc[i]["mjds_%i" % bi]

		#print mjds
		#print mjds[detecteds]
		
		lc = pycs.gen.lc.factory(jds=mjds, mags=mags, magerrs=magerrs)
		#lc_detected = pycs.gen.lc.factory(jds=mjds[detecteds], mags=mags[detecteds], magerrs=magerrs[detecteds])
		#lc_detected.plotcolour = "blue"
	
		# really rough test, we just fit a spline with chosen knotstep
		spl = pycs.gen.spl.fit(lcs=[lc], knotstep=20.0, stab=True,
	stabext=100.0, bokit=1, bokeps=2.0, boktests=5, verbose=False)	
		#spl_detected = pycs.gen.spl.fit(lcs=[lc_detected], knotstep=20.0, stab=True, stabext=100.0, bokit=1, bokeps=2.0, boktests=5, verbose=False)	

		#print spl.c
		#print spl.t
		#print len(mags[detecteds])
		samples.iloc[i]["spl_t_%i" % bi] = spl.t
		samples.iloc[i]["spl_c_%i" % bi] = spl.c
		if random.uniform(0, 1) > 0.999:	
			pycs.gen.lc.display([lc], [spl], knotsize=0.3)
		
	break	
writepickle(samples, "training_samples_spl.pkl")


NameError: name 'samples' is not defined